Import the required libraries

In [1]:
from jupyter_core.paths import jupyter_path

In [608]:
import warnings
warnings.filterwarnings('ignore')

In [637]:
import numpy as np
import pandas as pd
import seaborn as sns
import os
import glob
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import TimeSeriesSplit
from sklearn import metrics
from sklearn.metrics import mean_absolute_error as mae
from sklearn.metrics import accuracy_score, roc_auc_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from sklearn.metrics import precision_score
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import KFold
import lightgbm as lgb # Our ML library
from datetime import date
from datetime import datetime
from datetime import timedelta

In [4]:
#We will run the normal distribution test with
from scipy.stats import shapiro

In [103]:
#Cross validation libraries
from sklearn.model_selection import (
    train_test_split,
    TimeSeriesSplit,
    KFold,
    StratifiedKFold,
    GroupKFold,
    StratifiedGroupKFold,
)

In [5]:
#Can help with the visualization
import matplotlib.gridspec as gridspec
import SeabornFig2Grid as sfg

In [ ]:
import joblib

Read all CSVs with stocks data and append to one big file

record the dataframe to speed up the future reading process

os.chdir("C:/Users/oleg.kazanskyi/Personal-oleg.kazanskyi/Trading Python/")
df.to_csv("THE_FINAL_DATASET_2023.csv")

In [6]:
os.chdir("C:/Users/oleg.kazanskyi/Personal-oleg.kazanskyi/Trading Python")
df= pd.read_csv("THE_FINAL_DATASET_2023.csv")
os.chdir("C:/Users/oleg.kazanskyi/Personal-oleg.kazanskyi/Trading Python")
#df.drop(["Unnamed: 0","dataCode","name","description","statementType","units","Unnamed: 0.2", "Unnamed: 0.1"], axis = 1, inplace = True)
df = df[df.close.notna()]

In [7]:
df['YoY_DY'] = 100*df['DY'].pct_change(periods = -365)

In [8]:
df.shape

(903111, 60)

In [9]:
df.duplicated().sum()

0

Set up Date column as an index columns

In [10]:
df["date"] = pd.to_datetime(df["date"])

Let's check how much empty values we have by column

In [11]:
zeroes = df.isnull().sum()
print(zeroes[zeroes>0])
del zeroes

days_after_earnings_report    219898
sector                            35
industry                          35
LTDE                            4553
DE                              1208
DPR                           102932
Acc_Rec_Pay_Ration              1879
DY                            102932
PEG_Forward                     2277
PEG_Backwards                    128
EPS_surprise                  219898
EPS_YoY_Growth                219898
EPS_QoQ_frcst_diff            219898
EPS_1Y_exp_Change             222833
YoY_ROE                         9539
YoY_LTDE                        9539
YoY_DE                          9539
YoY_CR                          9539
YoY_GM                          9539
YoY_ROA                         9539
YoY_DPR                       100620
YoY_AR_Ration                  10965
YoY_ES                          9539
YoY_Piotroski                   9539
YoY_PE                          9539
YoY_PB                          9539
YoY_PEGF                        9539
Y

### Dealing with "EPS" column

There are many null values in the "current_ratio" column. Let's see the stocks where it happens and then decide what to do

In [12]:
df[df.EPS_surprise.isnull()].stock.unique()

array(['AMD', 'AMZN', 'APA', 'APTV', 'AVB', 'AZO', 'BAX', 'BBY', 'BIO',
       'BSX', 'CAG', 'CCL', 'CDNS', 'CHD', 'COST', 'CZR', 'DLTR', 'ENPH',
       'ETSY', 'FOX', 'GILD', 'HSIC', 'INCY', 'INVH', 'JKHY', 'KHC', 'L',
       'LEN', 'MOS', 'NI', 'NOW', 'ORLY', 'PARA', 'PEP', 'PKG', 'PNW',
       'RJF', 'ROP', 'ROST', 'TAP', 'TSLA', 'TTWO', 'TXT', 'UNH', 'YUM'],
      dtype=object)

After checking the original files, it's apparently the null values comes from the older data.

Before 2017 our API did not provide us info about earinings. 

We can drop the reocrds with 'nan' values

we will remove only 10k records 

In [13]:
df = df[df.EPS_surprise.notnull()]

Let's check the dataset again

In [628]:
def check_null_cols(df):
    zeroes = df.isnull().sum()
    print(zeroes[zeroes>0])
    
check_null_cols(df)   

### Dealing with the null values in the "dividends" column

About 30% of data is affected by null values in the dividends columns.
Let's check how many companies are in the list

In [15]:
print("Number of companies with zero values in dividends",len(df[df['DY'].isnull()].stock.unique()))

Number of companies with zero values in dividends 49


This companies do not pay dividends, we can replace payments to "0"

In [16]:
df['DY'].fillna(0, inplace=True)
df['YoY_DPR'].fillna(0, inplace=True)
df['DPR'].fillna(0, inplace=True)
df['YoY_DY'].fillna(0, inplace=True)                   

### Dealing with YoY

The YoY variables with null values are caused by YoY calculation of the rows without historical data.
Let's drop these

In [17]:
df = df[df.YoY_CR.notnull()]

Let's check the dataset again

In [18]:
check_null_cols(df)   

sector                        35
industry                      35
LTDE                        4553
DE                          1208
Acc_Rec_Pay_Ration          1879
PEG_Forward                 2277
PEG_Backwards                119
EPS_1Y_exp_Change           2935
YoY_AR_Ration               1426
future_15dprice_change      5027
future_30dprice_change     10051
future_60dprice_change     20074
future_90dprice_change     30084
future_120dprice_change    40094
future_150dprice_change    50104
VIX_DoD                      459
VIX_WoW                     2295
VIX_MoM                    10095
dtype: int64


In [19]:
df.shape

(673674, 60)

### Dealing with debt ratio

Let's check the companies that contain null values

In [20]:
df[df['DE'].isnull()].stock.unique()

array(['FFIV', 'FTNT'], dtype=object)

It seems these companies had zero debt for some quarters.

Let's replace with "0"

In [21]:
df['DE'].fillna(0, inplace=True)
df['LTDE'].fillna(0, inplace=True)

In [22]:
check_null_cols(df)   

sector                        35
industry                      35
Acc_Rec_Pay_Ration          1879
PEG_Forward                 2277
PEG_Backwards                119
EPS_1Y_exp_Change           2935
YoY_AR_Ration               1426
future_15dprice_change      5027
future_30dprice_change     10051
future_60dprice_change     20074
future_90dprice_change     30084
future_120dprice_change    40094
future_150dprice_change    50104
VIX_DoD                      459
VIX_WoW                     2295
VIX_MoM                    10095
dtype: int64


### Dealing with VIX

Let's remove null values for VIX as these are related to the historical calculations

In [23]:
df = df[df.VIX_MoM.notnull()]

In [24]:
check_null_cols(df) 

sector                        13
industry                      13
Acc_Rec_Pay_Ration          1835
PEG_Forward                 2034
PEG_Backwards                 61
EPS_1Y_exp_Change           2692
YoY_AR_Ration               1404
future_15dprice_change      5016
future_30dprice_change     10023
future_60dprice_change     20033
future_90dprice_change     30043
future_120dprice_change    40053
future_150dprice_change    50063
dtype: int64


### Dealing with Accounts Payable

In [25]:
df[df['Acc_Rec_Pay_Ration'].isnull()].stock.unique()

array(['CINF', 'MAA'], dtype=object)

It seems these companies had zero Accounts receivables for some quarters

In [26]:
df['Acc_Rec_Pay_Ration'].fillna(0, inplace=True)

### Dealing with PEG

In [27]:
df[df['PEG_Forward'].isnull()].stock.unique()

array(['FOXA', 'LHX', 'PLD', 'SEDG'], dtype=object)

We can drop the rows as the effect would be unsignificant

In [28]:
df = df[df.PEG_Forward.notnull()]

In [29]:
df[df['PEG_Backwards'].isnull()].stock.unique()

array(['PKI', 'SYF'], dtype=object)

In [30]:
df = df[df.PEG_Backwards.notnull()]

In [31]:
df = df[df.EPS_1Y_exp_Change.notnull()]

In [32]:
check_null_cols(df) 

sector                        13
industry                      13
YoY_AR_Ration               1404
future_15dprice_change      5016
future_30dprice_change     10023
future_60dprice_change     20033
future_90dprice_change     30043
future_120dprice_change    40053
future_150dprice_change    50063
dtype: int64


### Dealing with Industry

In [33]:
df[df['sector'].isnull()].stock.unique()

array(['GEN'], dtype=object)

Let's fill the sector and industry values for the GEN company

In [34]:
df.loc[df['stock'] == 'GEN','sector'] = 'Information Technology'
df.loc[df['stock'] == 'GEN','industry'] = 'Software & Services'

In [35]:
check_null_cols(df) 

YoY_AR_Ration               1404
future_15dprice_change      5016
future_30dprice_change     10023
future_60dprice_change     20033
future_90dprice_change     30043
future_120dprice_change    40053
future_150dprice_change    50063
dtype: int64


In [36]:
df = df[df.YoY_AR_Ration.notnull()]

### Working with data types

Now we can keep only the necessery columns

In [37]:
categoric_columns = df.select_dtypes(include='object').columns
for col in categoric_columns:
    if col == "stock":
        continue
    print(f"column {col}, data: \n {df[col].unique()}")

column sector, data: 
 ['Health Care' 'Industrials' 'Consumer Discretionary'
 'Information Technology' 'Financials' 'Consumer Staples' 'Utilities'
 'Materials' 'Real Estate' 'Energy' 'Communication Services']
column industry, data: 
 ['Pharmaceuticals, Biotechnology & Life Sciences' 'Transportation'
 'Retailing' 'Technology Hardware & Equipment'
 'Health Care Equipment & Services' 'Insurance' 'Software & Services'
 'Semiconductors & Semiconductor Equipment' 'Food, Beverage & Tobacco'
 'Utilities' 'Materials' 'Capital Goods' 'Diversified Financials'
 'Real Estate' 'Energy' 'Automobiles & Components' 'Media & Entertainment'
 'Banks' 'Consumer Services' 'Household & Personal Products'
 'Food & Staples Retailing' 'Commercial & Professional Services'
 'Consumer Durables & Apparel' 'Telecommunication Services']


In [38]:
#Number of stocks per industry
df[['sector','industry','stock']].groupby('industry').stock.nunique()

industry
Automobiles & Components                           5
Banks                                             18
Capital Goods                                     48
Commercial & Professional Services                 9
Consumer Durables & Apparel                       13
Consumer Services                                 15
Diversified Financials                            26
Energy                                            23
Food & Staples Retailing                           4
Food, Beverage & Tobacco                          22
Health Care Equipment & Services                  36
Household & Personal Products                      6
Insurance                                         23
Materials                                         29
Media & Entertainment                             20
Pharmaceuticals, Biotechnology & Life Sciences    27
Real Estate                                       29
Retailing                                         22
Semiconductors & Semiconductor Equipm

In [39]:
#Number of stocks per sector
df[['sector','industry','stock']].groupby('sector').stock.nunique()

sector
Communication Services    24
Consumer Discretionary    55
Consumer Staples          32
Energy                    23
Financials                67
Health Care               63
Industrials               71
Information Technology    73
Materials                 29
Real Estate               29
Utilities                 29
Name: stock, dtype: int64

We can drop "Stocks" and "Industry" columns as there are too many unique values that block us from generalizing the data. 

In [40]:
#df.drop(["industry", "stock"], axis = 1, inplace = True)
#df.drop(["industry"], axis = 1, inplace = True)

Let's also check if there are any infinite numbers that can cause same trublesas nan

In [41]:
df.replace([np.inf, -np.inf], np.nan, inplace = True)

check_null_cols(df) 

EPS_YoY_Growth                36
EPS_QoQ_frcst_diff           101
EPS_1Y_exp_Change            206
future_15dprice_change      5005
future_30dprice_change     10001
future_60dprice_change     19989
future_90dprice_change     29977
future_120dprice_change    39965
future_150dprice_change    49953
dtype: int64


In [42]:
df = df[df.EPS_1Y_exp_Change.notnull() & df.EPS_YoY_Growth.notnull() & df.EPS_QoQ_frcst_diff.notnull()]

In [43]:
check_null_cols(df)

future_15dprice_change      5005
future_30dprice_change     10001
future_60dprice_change     19989
future_90dprice_change     29977
future_120dprice_change    39965
future_150dprice_change    49953
dtype: int64


### Data Cleaning is Over!

## Trimming the data to avoid overfitting

 Our dataset have the range of dates that are very close to each other. 

The changes in trading value are rarely very different between today and yesterday. 

There are exemptions but mostly these are connected with big surprises, and in any case we would notice the change even if we track every other day or every 4th day.

It means that removing part of the dataset should help us in building more generalized model since we will be looking for trend but not for matching values.


Let's do this, let's remove 4/5 of the dataset.

That means we will keep only one day of data per week. It will help us to generalize the dataset

In [44]:
print("Old dataframe shape:", df.shape)
df_compact = df.iloc[::5, :]
print("New dataframe shape:", df_compact.shape)

Old dataframe shape: (659142, 59)
New dataframe shape: (131829, 59)


The final dataset is 131K rows long with 53 variables and 6 targets(price after 15 days, 30 days, 60 days, 90 days, 120 days and 150 days)

We can remove the bigger dataset, but let's save it to the file before doing so.

In [45]:
os.chdir("C:/Users/oleg.kazanskyi/Personal-oleg.kazanskyi/Trading Python/ML_Part")
df.to_csv("full_cleaned_dataframe_2023.csv", index = False, header = True)
del df

Lets' save the shorter version of the dataframe as well so we can get it faster when required

In [ ]:
df_compact.to_csv("shorter_cleaned_dataframe_2023.csv", index = False, header = True)